In [1]:
!pip install transformers datasets torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
from google.colab import files

# Upload the CSV file
uploaded = files.upload()

# Load the uploaded file into a DataFrame
df = pd.read_csv(next(iter(uploaded)))

# Map labels to integers (Functional: 1, Non-Functional: 0)
label_mapping = {'F': 1, 'NF': 0}
df['labels'] = df['RequirementType'].map(label_mapping)

# Split the dataset into train and test
train_df, test_df = train_test_split(df[['content', 'labels']], test_size=0.2, random_state=42)

# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)


Saving final_corrected_fine_labeled_reviews.csv to final_corrected_fine_labeled_reviews.csv


In [3]:
from transformers import DistilBertTokenizer

# Load pre-trained DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples['content'], padding="max_length", truncation=True)

# Tokenize datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/9996 [00:00<?, ? examples/s]

Map:   0%|          | 0/2499 [00:00<?, ? examples/s]

In [4]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

# Load DistilBERT model for sequence classification
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Fine-tune the model
trainer.train()


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.086900,0.056056
2,0.020300,0.018805
3,0.011900,0.017508


TrainOutput(global_step=3750, training_loss=0.07124004465738933, metrics={'train_runtime': 1678.3511, 'train_samples_per_second': 17.868, 'train_steps_per_second': 2.234, 'total_flos': 3972432350896128.0, 'train_loss': 0.07124004465738933, 'epoch': 3.0})

In [5]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Step 1: Get predictions from the model on the test set
predictions = trainer.predict(test_dataset)

# Step 2: Convert logits to predicted class
preds = np.argmax(predictions.predictions, axis=-1)

# Step 3: Calculate accuracy
accuracy = accuracy_score(test_df['labels'], preds)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

# Step 4: Calculate precision, recall, and F1-score
precision = precision_score(test_df['labels'], preds, average='weighted')
recall = recall_score(test_df['labels'], preds, average='weighted')
f1 = f1_score(test_df['labels'], preds, average='weighted')

# Print Precision, Recall, F1-score
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

# Step 5: Optional - Print full classification report for more detailed metrics
print("Classification Report:\n")
print(classification_report(test_df['labels'], preds, target_names=['Non-Functional', 'Functional'], zero_division=1))


Model Accuracy: 99.56%
Precision: 1.00
Recall: 1.00
F1-Score: 1.00
Classification Report:

                precision    recall  f1-score   support

Non-Functional       1.00      1.00      1.00      1402
    Functional       0.99      1.00      0.99      1097

      accuracy                           1.00      2499
     macro avg       1.00      1.00      1.00      2499
  weighted avg       1.00      1.00      1.00      2499



In [6]:
# Save the fine-tuned model
trainer.save_model('./app_review_model')

# Save the tokenizer files
tokenizer.save_pretrained('./app_review_model')


('./app_review_model/tokenizer_config.json',
 './app_review_model/special_tokens_map.json',
 './app_review_model/vocab.txt',
 './app_review_model/added_tokens.json')

In [7]:
!pip install huggingface_hub


In [8]:
from huggingface_hub import notebook_login

notebook_login()


In [9]:
from huggingface_hub import HfApi

# Upload the entire directory to Hugging Face
api = HfApi()
api.upload_folder(
    folder_path='./app_review_model',  # Path to the folder with the model and tokenizer files
    repo_id='RafidMehda/app_review_model',  # Your model repository on Hugging Face
    repo_type='model'
)


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/RafidMehda/app_review_model/commit/0146ba2ca8ccbfec350377cfebca9a83cb17d7cf', commit_message='Upload folder using huggingface_hub', commit_description='', oid='0146ba2ca8ccbfec350377cfebca9a83cb17d7cf', pr_url=None, pr_revision=None, pr_num=None)